# Documentation

## Interesting metrics

- Average, Min, Max operating hours
- Earliest starting time
- Latest ending time
- Bucket to breakfast, lunch, dinner, supper
- Max closed day of week
- Group based on name similarity
- Group based on when they operate
- Missing ending time
- Open on weekends

## Use Cases

- What's open right now 
- Search based on name of restaurant (exact and fuzzy)
- Filter based on day of week, weekends , operating time

## Improvements

- Error handling when parsing CSV file
- Write metrics to DuckDB and use it for querying
- Timezone handling

## Scalable Solution

- Upload CSV files to S3
- Run flink job to watch files on S3 directory
- Incremental update to Elasticsearch / Druid 
- Superset on Elasticsearch / Druid

## Database Access Pattern

- Store day of week as bitstring to indicate open or close


# Setup

In [1]:
!pip install "modin[all]"
!pip install pandas==1.4.2
!pip install ipyfilechooser
!pip install duckdb==0.3.4
!pip install pyarrow
!pip install swifter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

     |████████████████████████████████| 853 kB 2.4 MB/s            
     |████████████████████████████████| 11.7 MB 4.5 MB/s            
     |████████████████████████████████| 136 kB 5.4 MB/s            
     |████████████████████████████████| 1.1 MB 1.4 MB/s            
     |████████████████████████████████| 1.8 MB 6.4 MB/s            
     |████████████████████████████████| 68 kB 4.7 MB/s             
     |████████████████████████████████| 52.9 MB 148 kB/s            
     |████████████████████████████████| 830 kB 2.5 MB/s            
     |████████████████████████████████| 117 kB 1.3 MB/s            
     |████████████████████████████████| 283 kB 1.8 MB/s            
     |████████████████████████████████| 158 kB 2.2 MB/s            
     |████████████████████████████████| 8.8 MB 3.3 MB/s            
     |████████████████████████████████| 4.1 MB 2.5 MB/s            
     |████████████████████████████████| 58 kB 3.5 MB/s            
     |████████████████████████████████| 3.1 MB 

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 58 kB 2.8 MB/s            
     |████████████████████████████████| 114 kB 2.6 MB/s            
     |████████████████████████████████| 461 kB 3.9 MB/s            
     |████████████████████████████████| 211 kB 3.1 MB/s            
     |████████████████████████████████| 156 kB 4.6 MB/s            


     |████████████████████████████████| 155 kB 4.2 MB/s            
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
Using legacy 'setup.py install' for gpustat, since package 'wheel' is not installed.
Using legacy 'setup.py install' for nvidia-ml-py3, since package 'wheel' is not installed.
    Running setup.py install for nvidia-ml-py3 ... done
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Uninstalling pandas-1.2.5:
      Successfully uninstalled pandas-1.2.5
    Running setup.py install for gpustat ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-superset 1.3.1 requires pandas<1.3,>=1.2.2, but you have pandas 1.4.2 which is incompatible.
You should consider upgrading via the '/home/gin/.pyenv/versions/3.8.4/envs/datascience/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgra

You should consider upgrading via the '/home/gin/.pyenv/versions/3.8.4/envs/datascience/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 13.9 MB 153 kB/s            
You should consider upgrading via the '/home/gin/.pyenv/versions/3.8.4/envs/datascience/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/gin/.pyenv/versions/3.8.4/envs/datascience/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 647 kB 2.2 MB/s            
  Preparing metadata (setup.py) ... done


Using legacy 'setup.py install' for swifter, since package 'wheel' is not installed.
    Running setup.py install for swifter ... done
You should consider upgrading via the '/home/gin/.pyenv/versions/3.8.4/envs/datascience/bin/python3.8 -m pip install --upgrade pip' command.


# Import

In [2]:
import modin.pandas as pd
import datetime
import itertools
from ipyfilechooser import FileChooser
import duckdb
import pyarrow as pa
from pyarrow import csv
import pyarrow.dataset as ds
import swifter
from collections import Counter

# Widgets

## CSV

In [3]:
# Create and display a FileChooser widget
csv_chooser = FileChooser('.')
csv_chooser.title = '<b>Upload CSV</b>'
csv_chooser.filter_pattern = "*.csv"
display(csv_chooser)

FileChooser(path='/home/gin/projects/personal/restaurant_status', filename='', title='<b>Upload CSV</b>', show…

## DB

In [4]:
# Create and display a FileChooser widget
db_chooser = FileChooser('.')
db_chooser.title = '<b>Upload DB (optional)</b>'
db_chooser.filter_pattern = "*.csv"
display(db_chooser)

FileChooser(path='/home/gin/projects/personal/restaurant_status', filename='', title='<b>Upload DB (optional)<…

# Input Resolution

In [6]:
DEFAULT_FILE_PATH = './data/input/dinning_places_open_hrs.csv'
input_file_path = csv_chooser.selected or DEFAULT_FILE_PATH
db_file_path = db_chooser.selected or 'duck.db'
db_conn = duckdb.connect(db_file_path)
read_options = csv.ReadOptions(
               column_names=["dining_place_name", "opening_time"],
               skip_rows=1)
input_csv = csv.read_csv(input_file_path, read_options)
df = input_csv.to_pandas()
# Create table based on raw Panda Dataframe
db_conn.execute('CREATE TABLE IF NOT EXISTS raw_dinning_places AS SELECT * FROM df')

# EDA

In [13]:
db_conn.query('SELECT COUNT(*) FROM raw_dinning_places')

---------------------
-- Expression Tree --
---------------------
Subquery

---------------------
-- Result Columns  --
---------------------
- count_star() (BIGINT)

---------------------
-- Result Preview  --
---------------------
count_star()	
BIGINT	
[ Rows: 1]
50	



# Functions

In [37]:
dow_map = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

def parse_timing(timing):
    parsed_timing = []
    try:
        if (not timing):
            return ""
        time = timing.strip().split('-')
        for t in time:
            parsed_time = t.strip().split(' ')
            unit_time = parsed_time[0]
            hour = unit_time.split(":")[0]
            minute = "00"
            if (":" in unit_time):
                minute = unit_time.split(":")[-1]
            hasPM = len(parsed_time) > 1 and ("pm" in parsed_time[1] or "PM" in parsed_time[1])
            if (hasPM and int(hour) != 12):
                hour = str(int(hour) + 12)
            operating_timing = hour.zfill(2) + minute if hour else ''
            parsed_timing.append(operating_timing)
    except Exception as err:
        print("Error parsing timing", err)
    finally:
        return '-'.join(parsed_timing)
            

def process_section(section, opening_time):
    try:
        operating_time = section[-1].strip().split(" ", 1)[-1]
        # print("Time", operating_time)
        for i in section:
            days = i.strip().split(" ")[0].split("-")
            # print(days)
            if (len(days) > 1) and operating_time:
                start_idx = dow_map.index(days[0].lower())
                end_idx = dow_map.index(days[-1].lower()) + 1
                opening_time[start_idx:end_idx] = [operating_time] * (end_idx - start_idx)
            elif operating_time:
                opening_time[dow_map.index(days[0].lower())] = operating_time
        return opening_time
    except Exception as err:
        print("Error processing section", err)
        return opening_time
    
    
def parse_time(row):
    # print("Row", row)
    operating_time = [''] * 7
    blocks = row.strip().split("/")
    sections = list(itertools.chain(*[x.strip().split(",") for x in blocks]))
    sections = [process_section(x.strip().split(","), operating_time) for x in blocks]
    return sections[-1]

def parse_start_end_time(period):
    ranges = period.split('-')
    return [ranges[0], ranges[1]] if len(ranges) > 1 else [ranges[0], '']

def update_operating_time(df, dow_map):
    for idx, day in enumerate(dow_map):
        df[f"{day}_start_time"] = df.swifter.apply(lambda x: parse_start_end_time(x['parsed_timing'].split(',')[idx])[0], axis=1)
        df[f"{day}_end_time"] = df.swifter.apply(lambda x: parse_start_end_time(x['parsed_timing'].split(',')[idx])[1], axis=1)
    return df 

def update_operating_time_metrics(df):
    for row in df.itertuples():
        operating_time_freq = Counter(getattr(row, 'parsed_timing').split(','))
        most_common_operating_time = operating_time_freq.most_common(1)[0][0]
        start_time, end_time = parse_start_end_time(most_common_operating_time)
        print(getattr(row, 'dining_place_name'), start_time, end_time)
        start_datetime = datetime.datetime.strptime(start_time, '%H%M')
        end_datetime = datetime.datetime.strptime('0000' if end_time == '' else end_time, '%H%M')
        if (end_time != '' and int(end_time) < int(start_time)):
            start_datetime = start_datetime.replace(day=1)
            end_datetime = end_datetime.replace(day=2)
        duration_in_hrs = (end_datetime - start_datetime).total_seconds() / 3600
        breakfast = int(start_time) >= int('0600') and int(start_time) < int('1000') and int(end_time) >= int('1100')
        lunch = int(start_time) < int('1300') and int(end_time) > int('1400')
        dinner = int(start_time) < int('1900') and int(end_time) > int('2100')
        supper = int(end_time) >= int('2200') or (int(end_time) < int(start_time))
        df.at[row.Index, 'most_common_operating_time'] = most_common_operating_time
        df.at[row.Index, 'duration_in_hrs'] = duration_in_hrs
        df.at[row.Index, 'breakfast'] = breakfast
        df.at[row.Index, 'lunch'] = lunch
        df.at[row.Index, 'dinner'] = dinner
        df.at[row.Index, 'supper'] = supper

def get_closed_days(df, dow_map):
    return [len(df.loc[(df[f"{i}_start_time"] == "") & (df[f"{i}_start_time"] == "")]) for i in dow_map]
        

# Processing

In [12]:
# Attempt 1: Parse opening_time to duration for each day of 
df['parsed_timing'] = df.swifter.apply(lambda x: ','.join([parse_timing(i) for i in parse_time(x['opening_time'])]), axis=1)
df['opening_days'] = df.swifter.apply(lambda x: ''.join(['0' if i == "" else '1' for i in x['parsed_timing'].split(',')]), axis=1)
# Generate start time and end time for each day of week
df = update_operating_time(df, dow_map)
df

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/50 [00:00<?, ?it/s]

,dining_place_name,opening_time,parsed_timing,opening_days,mon_start_time,mon_end_time,tue_start_time,tue_end_time,wed_start_time,wed_end_time,...,sat_start_time,sat_end_time,sun_start_time,sun_end_time,most_common_operating_time,duration_in_hrs,breakfast,lunch,dinner,supper
0,Osakaya Restaurant,"Mon-Thu, Sun 11:30 am - 9 pm / Fri-Sat 11:30 ...","1130-2100,1130-2100,1130-2100,1130-2100,1130-2...",1111111,1130,2100,1130,2100,1130,2100,...,1130,2130,1130,2100,1130-0900,21.5,False,False,False,True
1,The Stinking Rose,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30...","1130-2200,1130-2200,1130-2200,1130-2200,1130-2...",1111111,1130,2200,1130,2200,1130,2200,...,1130,2300,1130,2200,1130-1000,22.5,False,False,False,True
2,McCormick & Kuleto's,"Mon-Thu, Sun 11:30 am - 10 pm / Fri-Sat 11:30...","1130-2200,1130-2200,1130-2200,1130-2200,1130-2...",1111111,1130,2200,1130,2200,1130,2200,...,1130,2300,1130,2200,1130-1000,22.5,False,False,False,True
3,Mifune Restaurant,Mon-Sun 11 am - 10 pm,"1100-2200,1100-2200,1100-2200,1100-2200,1100-2...",1111111,1100,2200,1100,2200,1100,2200,...,1100,2200,1100,2200,1100-1000,23.0,False,False,False,True
4,The Cheesecake Factory,Mon-Thu 11 am - 11 pm / Fri-Sat 11 am - 12:30...,"1100-2300,1100-2300,1100-2300,1100-2300,1100-1...",1111111,1100,2300,1100,2300,1100,2300,...,1100,1230,1000,2300,1100-1100,0.0,False,False,False,False
5,New Delhi Indian Restaurant,Mon-Sat 11:30 am - 10 pm / Sun 5:30 pm - 10 pm,"1130-2200,1130-2200,1130-2200,1130-2200,1130-2...",1111111,1130,2200,1130,2200,1130,2200,...,1130,2200,1730,2200,1130-1000,22.5,False,False,False,True
6,Iroha Restaurant,"Mon-Thu, Sun 11:30 am - 9:30 pm / Fri-Sat 11:...","1130-2130,1130-2130,1130-2130,1130-2130,1130-2...",1111111,1130,2130,1130,2130,1130,2130,...,1130,2200,1130,2130,1130-0930,22.0,False,False,False,True
7,Rose Pistola,Mon-Thu 11:30 am - 10 pm / Fri-Sun 11:30 am -...,"1130-2200,1130-2200,1130-2200,1130-2200,1130-2...",1111111,1130,2200,1130,2200,1130,2200,...,1130,2300,1130,2300,1130-1000,22.5,False,False,False,True
8,Alioto's Restaurant,Mon-Sun 11 am - 11 pm,"1100-2300,1100-2300,1100-2300,1100-2300,1100-2...",1111111,1100,2300,1100,2300,1100,2300,...,1100,2300,1100,2300,1100-1100,0.0,False,False,False,False
9,Canton Seafood & Dim Sum Restaurant,Mon-Fri 10:30 am - 9:30 pm / Sat-Sun 10 am - ...,"1030-2130,1030-2130,1030-2130,1030-2130,1030-2...",1111111,1030,2130,1030,2130,1030,2130,...,1000,2130,1000,2130,1030-0930,23.0,False,False,False,True


In [13]:
update_operating_time_metrics(df)

Osakaya Restaurant 1130 2100
The Stinking Rose 1130 2200
McCormick & Kuleto's 1130 2200
Mifune Restaurant 1100 2200
The Cheesecake Factory 1100 2300
New Delhi Indian Restaurant 1130 2200
Iroha Restaurant 1130 2130
Rose Pistola 1130 2200
Alioto's Restaurant 1100 2300
Canton Seafood & Dim Sum Restaurant 1030 2130
All Season Restaurant 1000 2130
Bombay Indian Restaurant 1130 2230
Sam's Grill & Seafood Restaurant 1100 2100
2G Japanese Brasserie 1100 2200
Restaurant Lulu 1130 2100
Sudachi 1700 1230
Hanuri 1100 1200
Herbivore 0900 2200
Penang Garden 1100 2200
John's Grill 1100 2200
Quan Bac 1100 2200
Bamboo Restaurant 1100 1200
Burger Bar 1100 2200
Blu Restaurant 1130 2200
Naan 'N' Curry 1100 0400
Shanghai China Restaurant 1100 2130
Tres 1130 2200
Isobune Sushi 1130 2130
Viva Pizza Restaurant 1100 1200
Far East Cafe 1130 2200
Parallel 37 1130 2200
Bai Thong Thai Cuisine 1100 2300
Alhamra 1100 2300
A-1 Cafe Restaurant 1100 2200
Nick's Lighthouse 1100 2230
Paragon Restaurant & Bar 1130 2200
Ch

# Stats

In [38]:
# Count number of empty start time and end time for particular day of week
get_closed_days(df, dow_map)
# Calculate max duration_in_hrs
df['duration_in_hrs'].max()
# Calculate min duration_in_hrs
df['duration_in_hrs'].min()
# Calculate avg duration_in_hrs
df['duration_in_hrs'].mean()
# Number of restaurants open for breakfast, lunch, dinner, supper
df['breakfast'].value_counts()
df['lunch'].value_counts()
df['dinner'].value_counts()
df['supper'].value_counts()

[0, 1, 0, 0, 0, 0, 3]

19.5

1.0

10.81

False    47
True      3
Name: breakfast, dtype: int64

True     43
False     7
Name: lunch, dtype: int64

True     40
False    10
Name: dinner, dtype: int64

True     38
False    12
Name: supper, dtype: int64

In [96]:
# Write output
df.to_csv('./data/output/dinning_places_open_hrs_processed.csv', index=False)

# Dashboard

In [170]:
import ipywidgets as widgets
import ipydatetime


day_selector = widgets.SelectMultiple(
    options=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
    description='Day of Week',
    disabled=False
)
start_time_picker = ipydatetime.TimePicker()
end_time_picker = ipydatetime.TimePicker()
is_open_now = widgets.Checkbox(
    value=False,
    description="Show me what's open now",
    disabled=False,
    indent=False
)

is_open_now
day_selector
start_time_picker
end_time_picker

Checkbox(value=False, description="Show me what's open now", indent=False)

SelectMultiple(description='Day of Week', options=('Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'), value=())

TimePicker(value=None, step=60.0)

TimePicker(value=None, step=60.0)

In [159]:
filtered_day = [dow_map.index(x.lower()) for x in list(day_selector.value)]
filtered_start_time = start_time_picker.value.strftime("%H%M") if start_time_picker.value else ""
filtered_end_time = end_time_picker.value.strftime("%H%M") if end_time_picker.value else ""

In [185]:
filtered_day
filtered_start_time
filtered_end_time
is_open_now.value

[2]

'1230'

''

True

In [246]:
def filter_row(row, is_open_now, filtered_day, start_time, end_time):
    today = datetime.datetime.today()
    current_dow = today.weekday()
    current_time = today.strftime('%H%M')
    timing = row['parsed_timing']
    matched_day = True
    matched_time = True
    if (is_open_now):
        filtered_day = [current_dow]
        start_time = current_time
    if len(filtered_day) > 0:
        open_dining = [timing.split(',')[x] for x in filtered_day]
        matched_day = '' not in open_dining
        print('Matched Day', matched_day, filtered_day)
        if (matched_day):
            start_can = True
            end_can = True
            for i in filtered_day:
                opening_time = timing.split(',')[i]
                start = opening_time.split('-')[0]
                end = opening_time.split('-')[1] if len(opening_time.split('-')) > 1 else ''
                print(parse_time(row['opening_time']), row['opening_time'], timing, opening_time, start, start_time)
                if (start_time != '' and start != '' and start_time < start):
                    start_can =  False
                if (end_time != '' and end != '' and end_time > end):
                    end_can = False
            matched_time = start_can and end_can
        else:
            return False
    print(matched_day, matched_time)
    return matched_day and matched_time
        
    

In [247]:
filtered_rows = df.apply(lambda x: filter_row(x, is_open_now.value, filtered_day, filtered_start_time, filtered_end_time), axis=1)
df[filtered_rows]

Matched Day True [0]
['11:30 am - 9 pm', '11:30 am - 9 pm', '11:30 am - 9 pm', '11:30 am - 9 pm', '11:30 am - 9:30 pm', '11:30 am - 9:30 pm', '11:30 am - 9 pm'] Mon-Thu, Sun 11:30 am - 9 pm  / Fri-Sat 11:30 am - 9:30 pm 1130-2100,1130-2100,1130-2100,1130-2100,1130-2130,1130-2130,1130-2100 1130-2100 1130 0000
True False
Matched Day True [0]
['11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 11 pm', '11:30 am - 11 pm', '11:30 am - 10 pm'] Mon-Thu, Sun 11:30 am - 10 pm  / Fri-Sat 11:30 am - 11 pm 1130-2200,1130-2200,1130-2200,1130-2200,1130-2300,1130-2300,1130-2200 1130-2200 1130 0000
True False
Matched Day True [0]
['11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 10 pm', '11:30 am - 11 pm', '11:30 am - 11 pm', '11:30 am - 10 pm'] Mon-Thu, Sun 11:30 am - 10 pm  / Fri-Sat 11:30 am - 11 pm 1130-2200,1130-2200,1130-2200,1130-2200,1130-2300,1130-2300,1130-2200 1130-2200 1130 0000
True False
Matched Day True [0]
['11 am - 10 pm'

,dining_place_name,opening_time,parsed_timing
